In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx, to_undirected, train_test_split_edges, negative_sampling
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
import gc

filtered_df = pd.read_csv('drive/MyDrive/TFM/GeneRIF/interactions_human_reduced.csv')
df = filtered_df[['gene_id', 'interactant_id']].drop_duplicates().sort_values(by=['gene_id', 'interactant_id'])
graph_df = df.loc[df['gene_id'] != df['interactant_id']]

deep_df = pd.read_csv('drive/MyDrive/definitivo/node2vec_embeddings3.csv').drop('Unnamed: 0', axis=1)  # p = q = 1
nod1_df = pd.read_csv('drive/MyDrive/definitivo/node2vec_embeddings4.csv').drop('Unnamed: 0', axis=1)  # p = 0.5 ; q = 2
nod2_df = pd.read_csv('drive/MyDrive/definitivo/node2vec_embeddings5.csv').drop('Unnamed: 0', axis=1)  # p = 2 ; q = 0.5

unique_genes = pd.concat([graph_df['gene_id'], graph_df['interactant_id']]).unique()
gene_to_index = {gene: i for i, gene in enumerate(unique_genes)}
index_to_gene = {i: gene for gene, i in gene_to_index.items()}

In [9]:
from torch_geometric.transforms import RandomLinkSplit

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [10]:
G = nx.Graph()
total = len(graph_df)
iteration = 0

for index, row in graph_df.iterrows():
  iteration += 1
  print(f"Processing row {iteration}/{total}")

  gene_id = row['gene_id']
  interactant_id = row['interactant_id']

  gene_index = gene_to_index[gene_id]
  interactant_index = gene_to_index[interactant_id]

  G.add_edge(gene_index, interactant_index)


# num_nodes = 20
# probability_of_edge_creation = 0.2
# G = nx.erdos_renyi_graph(num_nodes, probability_of_edge_creation)

Se han truncado las últimas 5000 líneas del flujo de salida.
Processing row 833505/838504
Processing row 833506/838504
Processing row 833507/838504
Processing row 833508/838504
Processing row 833509/838504
Processing row 833510/838504
Processing row 833511/838504
Processing row 833512/838504
Processing row 833513/838504
Processing row 833514/838504
Processing row 833515/838504
Processing row 833516/838504
Processing row 833517/838504
Processing row 833518/838504
Processing row 833519/838504
Processing row 833520/838504
Processing row 833521/838504
Processing row 833522/838504
Processing row 833523/838504
Processing row 833524/838504
Processing row 833525/838504
Processing row 833526/838504
Processing row 833527/838504
Processing row 833528/838504
Processing row 833529/838504
Processing row 833530/838504
Processing row 833531/838504
Processing row 833532/838504
Processing row 833533/838504
Processing row 833534/838504
Processing row 833535/838504
Processing row 833536/838504
Processing 

In [11]:
gene_csv = pd.read_csv('drive/MyDrive/TFM/data/gene_information_csv').set_index('gene_id').drop(columns=['pos_min','pos_max'], axis=1)
gene_csv.index = gene_csv.index.astype(str)

dnabert3_df = pd.read_parquet('drive/MyDrive/TFM/data/embedding_kmer3.parquet').set_index('gene_id')
dnabert4_df = pd.read_parquet('drive/MyDrive/TFM/data/embedding_kmer4.parquet').set_index('gene_id')
dnabert5_df = pd.read_parquet('drive/MyDrive/TFM/data/embedding_kmer5.parquet').set_index('gene_id')
dnabert6_df = pd.read_csv('drive/MyDrive/TFM/data/embedding_dnabert6.csv').drop(columns=['Unnamed: 0']).rename(columns={'gene': 'gene_id'}).set_index('gene_id')

dnabert3_df.index = dnabert3_df.index.astype(str)
dnabert4_df.index = dnabert4_df.index.astype(str)
dnabert5_df.index = dnabert5_df.index.astype(str)
dnabert6_df.index = dnabert6_df.index.astype(str)

gdnabert3_df = pd.concat([gene_csv, dnabert3_df], axis=1, join='inner')
gdnabert4_df = pd.concat([gene_csv, dnabert4_df], axis=1, join='inner')
gdnabert5_df = pd.concat([gene_csv, dnabert5_df], axis=1, join='inner')
gdnabert6_df = pd.concat([gene_csv, dnabert6_df], axis=1, join='inner')

In [12]:
tdnabert3_df = pd.concat([gdnabert3_df, deep_df.T], axis=1, join='inner')

In [13]:
new_df = gdnabert3_df[['rel_min', 'rel_max', 'chr_chr1', 'chr_chr10', 'chr_chr11', 'chr_chr12', 'chr_chr13', 'chr_chr14', 'chr_chr15', 'chr_chr16', 'chr_chr17', 'chr_chr18', 'chr_chr19', 'chr_chr2', 'chr_chr20', 'chr_chr21', 'chr_chr22', 'chr_chr3', 'chr_chr4', 'chr_chr5', 'chr_chr6', 'chr_chr7', 'chr_chr8', 'chr_chr9', 'chr_chrM', 'chr_chrX', 'chr_chrY']]

In [14]:
data = from_networkx(G)

edge_index = torch.tensor(list(G.edges)).t().contiguous()

data = Data(edge_index=edge_index, num_nodes=len(G.nodes))

data.x = torch.stack([torch.tensor(gdnabert3_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

data.edge_index = to_undirected(data.edge_index)

transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(tdnabert3_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)

transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 1028, 514).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")

Epoch: 001, Loss: 2.0631, Train Acc: 0.3335, Val AUC: 0.7466, Test Acc: 0.6739, Test AUC: 0.7785
Epoch: 002, Loss: 0.9287, Train Acc: 0.3562, Val AUC: 0.7444, Test Acc: 0.6685, Test AUC: 0.7769
Epoch: 003, Loss: 0.9625, Train Acc: 0.3424, Val AUC: 0.7417, Test Acc: 0.6676, Test AUC: 0.7746
Epoch: 004, Loss: 0.9670, Train Acc: 0.3393, Val AUC: 0.7449, Test Acc: 0.6691, Test AUC: 0.7777
Epoch: 005, Loss: 0.8942, Train Acc: 0.3440, Val AUC: 0.7518, Test Acc: 0.6735, Test AUC: 0.7851
Epoch: 006, Loss: 0.8356, Train Acc: 0.3558, Val AUC: 0.7624, Test Acc: 0.6808, Test AUC: 0.7965
Epoch: 007, Loss: 0.7949, Train Acc: 0.3750, Val AUC: 0.7747, Test Acc: 0.6901, Test AUC: 0.8084
Epoch: 008, Loss: 0.7718, Train Acc: 0.3973, Val AUC: 0.7867, Test Acc: 0.6961, Test AUC: 0.8193
Epoch: 009, Loss: 0.7553, Train Acc: 0.4114, Val AUC: 0.7972, Test Acc: 0.6987, Test AUC: 0.8285
Epoch: 010, Loss: 0.7464, Train Acc: 0.4164, Val AUC: 0.8068, Test Acc: 0.7005, Test AUC: 0.8364
Epoch: 011, Loss: 0.7430, Trai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(new_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])
transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")


Epoch: 001, Loss: 0.6779, Train Acc: 0.3333, Val AUC: 0.8968, Test Acc: 0.6667, Test AUC: 0.9026
Epoch: 002, Loss: 0.7117, Train Acc: 0.3336, Val AUC: 0.8974, Test Acc: 0.6777, Test AUC: 0.9023
Epoch: 003, Loss: 0.6654, Train Acc: 0.3585, Val AUC: 0.8852, Test Acc: 0.6731, Test AUC: 0.8931
Epoch: 004, Loss: 0.6805, Train Acc: 0.3496, Val AUC: 0.8757, Test Acc: 0.6677, Test AUC: 0.8883
Epoch: 005, Loss: 0.6849, Train Acc: 0.3363, Val AUC: 0.8832, Test Acc: 0.6671, Test AUC: 0.8940
Epoch: 006, Loss: 0.6823, Train Acc: 0.3347, Val AUC: 0.8909, Test Acc: 0.6671, Test AUC: 0.8989
Epoch: 007, Loss: 0.6767, Train Acc: 0.3350, Val AUC: 0.8946, Test Acc: 0.6678, Test AUC: 0.9010
Epoch: 008, Loss: 0.6711, Train Acc: 0.3366, Val AUC: 0.8959, Test Acc: 0.6699, Test AUC: 0.9014
Epoch: 009, Loss: 0.6658, Train Acc: 0.3415, Val AUC: 0.8960, Test Acc: 0.6763, Test AUC: 0.9009
Epoch: 010, Loss: 0.6593, Train Acc: 0.3557, Val AUC: 0.8954, Test Acc: 0.6917, Test AUC: 0.8996
Epoch: 011, Loss: 0.6526, Trai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(gdnabert3_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)

transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 1028, 514).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")

Epoch: 001, Loss: 1.7884, Train Acc: 0.3342, Val AUC: 0.7466, Test Acc: 0.6735, Test AUC: 0.7759
Epoch: 002, Loss: 0.9356, Train Acc: 0.3546, Val AUC: 0.7458, Test Acc: 0.6696, Test AUC: 0.7757
Epoch: 003, Loss: 0.9231, Train Acc: 0.3449, Val AUC: 0.7469, Test Acc: 0.6686, Test AUC: 0.7771
Epoch: 004, Loss: 0.9150, Train Acc: 0.3423, Val AUC: 0.7543, Test Acc: 0.6708, Test AUC: 0.7841
Epoch: 005, Loss: 0.8654, Train Acc: 0.3481, Val AUC: 0.7643, Test Acc: 0.6767, Test AUC: 0.7937
Epoch: 006, Loss: 0.8141, Train Acc: 0.3641, Val AUC: 0.7733, Test Acc: 0.6859, Test AUC: 0.8027
Epoch: 007, Loss: 0.7742, Train Acc: 0.3882, Val AUC: 0.7796, Test Acc: 0.6947, Test AUC: 0.8090
Epoch: 008, Loss: 0.7524, Train Acc: 0.4098, Val AUC: 0.7845, Test Acc: 0.6978, Test AUC: 0.8135
Epoch: 009, Loss: 0.7467, Train Acc: 0.4169, Val AUC: 0.7896, Test Acc: 0.6972, Test AUC: 0.8180
Epoch: 010, Loss: 0.7427, Train Acc: 0.4151, Val AUC: 0.7959, Test Acc: 0.6979, Test AUC: 0.8237
Epoch: 011, Loss: 0.7377, Trai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(gdnabert4_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)

transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 1028, 514).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")

gc.collect()
torch.cuda.empty_cache()

Epoch: 001, Loss: 1.9241, Train Acc: 0.3336, Val AUC: 0.7454, Test Acc: 0.6735, Test AUC: 0.7773
Epoch: 002, Loss: 0.9230, Train Acc: 0.3547, Val AUC: 0.7444, Test Acc: 0.6690, Test AUC: 0.7770
Epoch: 003, Loss: 0.9202, Train Acc: 0.3430, Val AUC: 0.7406, Test Acc: 0.6678, Test AUC: 0.7735
Epoch: 004, Loss: 0.9259, Train Acc: 0.3395, Val AUC: 0.7438, Test Acc: 0.6697, Test AUC: 0.7765
Epoch: 005, Loss: 0.8772, Train Acc: 0.3445, Val AUC: 0.7515, Test Acc: 0.6746, Test AUC: 0.7842
Epoch: 006, Loss: 0.8212, Train Acc: 0.3576, Val AUC: 0.7597, Test Acc: 0.6822, Test AUC: 0.7926
Epoch: 007, Loss: 0.7878, Train Acc: 0.3766, Val AUC: 0.7667, Test Acc: 0.6913, Test AUC: 0.7992
Epoch: 008, Loss: 0.7636, Train Acc: 0.3978, Val AUC: 0.7716, Test Acc: 0.6977, Test AUC: 0.8041
Epoch: 009, Loss: 0.7508, Train Acc: 0.4141, Val AUC: 0.7762, Test Acc: 0.7001, Test AUC: 0.8086
Epoch: 010, Loss: 0.7442, Train Acc: 0.4187, Val AUC: 0.7815, Test Acc: 0.7020, Test AUC: 0.8138
Epoch: 011, Loss: 0.7407, Trai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(gdnabert5_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)

transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 1028, 514).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")

gc.collect()
torch.cuda.empty_cache()

Epoch: 001, Loss: 1.8547, Train Acc: 0.3338, Val AUC: 0.7443, Test Acc: 0.6742, Test AUC: 0.7744
Epoch: 002, Loss: 0.9155, Train Acc: 0.3561, Val AUC: 0.7461, Test Acc: 0.6701, Test AUC: 0.7769
Epoch: 003, Loss: 0.8925, Train Acc: 0.3463, Val AUC: 0.7439, Test Acc: 0.6687, Test AUC: 0.7749
Epoch: 004, Loss: 0.9132, Train Acc: 0.3419, Val AUC: 0.7464, Test Acc: 0.6712, Test AUC: 0.7776
Epoch: 005, Loss: 0.8784, Train Acc: 0.3484, Val AUC: 0.7519, Test Acc: 0.6781, Test AUC: 0.7838
Epoch: 006, Loss: 0.8233, Train Acc: 0.3666, Val AUC: 0.7598, Test Acc: 0.6885, Test AUC: 0.7918
Epoch: 007, Loss: 0.7843, Train Acc: 0.3924, Val AUC: 0.7673, Test Acc: 0.6979, Test AUC: 0.7996
Epoch: 008, Loss: 0.7597, Train Acc: 0.4154, Val AUC: 0.7751, Test Acc: 0.7017, Test AUC: 0.8066
Epoch: 009, Loss: 0.7510, Train Acc: 0.4247, Val AUC: 0.7821, Test Acc: 0.7017, Test AUC: 0.8131
Epoch: 010, Loss: 0.7460, Train Acc: 0.4242, Val AUC: 0.7880, Test Acc: 0.7024, Test AUC: 0.8188
Epoch: 011, Loss: 0.7379, Trai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

data.x = torch.stack([torch.tensor(gdnabert6_df.loc[str(index_to_gene[node])].tolist(), dtype=torch.float) for node in G.nodes()])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x.numpy()), dtype=torch.float)

transform = RandomLinkSplit(num_val=0.2, num_test=0.2, is_undirected=True, add_negative_train_samples=True, neg_sampling_ratio=0.5)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(train_data.num_node_features, 1028, 514).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss, out, edge_label

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return out, data.edge_label

def compute_metrics(output, labels):
    pred = (output > 0.5).float()

    accuracy = accuracy_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    precision = precision_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    recall = recall_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    f1 = f1_score(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())
    roc_auc = roc_auc_score(labels.cpu().detach().numpy(), output.cpu().detach().numpy())
    conf_matrix = confusion_matrix(labels.cpu().detach().numpy(), pred.cpu().detach().numpy())

    return accuracy, precision, recall, f1, roc_auc, conf_matrix

best_val_auc = final_test_metrics = None
for epoch in range(1, 100):
    loss, train_out, train_labels = train()

    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc, train_conf_matrix = compute_metrics(train_out.sigmoid(), train_labels)

    val_out, val_labels = test(val_data)
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc, val_conf_matrix = compute_metrics(val_out, val_labels)
    val_auc = val_roc_auc

    test_out, test_labels = test(test_data)
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc, test_conf_matrix = compute_metrics(test_out, test_labels)

    final_test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_accuracy:.4f}, Val AUC: {val_auc:.4f}, '
          f'Test Acc: {test_accuracy:.4f}, Test AUC: {test_roc_auc:.4f}')

if final_test_metrics is not None:
    print(f'Final Test Metrics - '
          f'Accuracy: {final_test_metrics[0]:.4f}, '
          f'Precision: {final_test_metrics[1]:.4f}, '
          f'Recall: {final_test_metrics[2]:.4f}, '
          f'F1-Score: {final_test_metrics[3]:.4f}, '
          f'ROC-AUC: {final_test_metrics[4]:.4f}')

print("\nConfusion Matrices:")
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Test Confusion Matrix:\n", test_conf_matrix)

print("\nROC-AUC Scores:")
print(f"Train ROC-AUC: {train_roc_auc:.4f}")
print(f"Validation ROC-AUC: {val_roc_auc:.4f}")
print(f"Test ROC-AUC: {test_roc_auc:.4f}")

gc.collect()
torch.cuda.empty_cache()

Epoch: 001, Loss: 1.7427, Train Acc: 0.3339, Val AUC: 0.7477, Test Acc: 0.6744, Test AUC: 0.7773
Epoch: 002, Loss: 0.9042, Train Acc: 0.3570, Val AUC: 0.7532, Test Acc: 0.6691, Test AUC: 0.7834
Epoch: 003, Loss: 0.9031, Train Acc: 0.3438, Val AUC: 0.7533, Test Acc: 0.6680, Test AUC: 0.7841
Epoch: 004, Loss: 0.9041, Train Acc: 0.3404, Val AUC: 0.7575, Test Acc: 0.6698, Test AUC: 0.7883
Epoch: 005, Loss: 0.8588, Train Acc: 0.3448, Val AUC: 0.7663, Test Acc: 0.6754, Test AUC: 0.7972
Epoch: 006, Loss: 0.8151, Train Acc: 0.3596, Val AUC: 0.7777, Test Acc: 0.6866, Test AUC: 0.8084
Epoch: 007, Loss: 0.7799, Train Acc: 0.3867, Val AUC: 0.7861, Test Acc: 0.6982, Test AUC: 0.8161
Epoch: 008, Loss: 0.7591, Train Acc: 0.4144, Val AUC: 0.7925, Test Acc: 0.7029, Test AUC: 0.8215
Epoch: 009, Loss: 0.7472, Train Acc: 0.4255, Val AUC: 0.7984, Test Acc: 0.7021, Test AUC: 0.8266
Epoch: 010, Loss: 0.7414, Train Acc: 0.4220, Val AUC: 0.8039, Test Acc: 0.7016, Test AUC: 0.8315
Epoch: 011, Loss: 0.7416, Trai